# Humpback Whale Identification Challenge

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

import pandas as pd
import os

In [3]:
torch.cuda.is_available()

False

In [4]:
torch.backends.cudnn.enabled

True

In [5]:
PATH = "data/"
sz=128
arch=resnet34
bs=28

## Exploring the dataset

In [7]:
! ls {PATH}

sample_submission.csv  test  test.zip  train  train.csv  train.zip


In [10]:
training_data = pd.read_csv(f'{PATH}train.csv')
training_data.head()

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [15]:
print('Number of traing images %s' % len(os.listdir(f'{PATH}train'))) 
print('Number of test images %s' % len(os.listdir(f'{PATH}test'))) 

Number of traing images 9850
Number of test images 15610


So, the number of test images is way more than number of traing images